In [1]:
# actual_m = 10
# actual_c = 20

x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
y = [2, 1, 2, 1, 7, 6, 7, 6, 12, 11, 12, 11]
y

[2, 1, 2, 1, 7, 6, 7, 6, 12, 11, 12, 11]

In [2]:
node_1_data = {"x": x[:4], "y": y[:4]}
node_2_data = {"x": x[4:8], "y": y[4:8]}
node_3_data = {"x": x[8:12], "y": y[8:12]}

node_data = [node_1_data, node_2_data, node_3_data]
node_data

[{'x': [1, 2, 3, 4], 'y': [2, 1, 2, 1]},
 {'x': [5, 6, 7, 8], 'y': [7, 6, 7, 6]},
 {'x': [9, 10, 11, 12], 'y': [12, 11, 12, 11]}]

In [3]:
y_mean = sum(y)/len(y)
y_mean
node_y_mean = [
    sum(y[:4])/4,
    sum(y[4:8])/4,
    sum(y[8:12])/4
]

In [4]:
node_params = [
    {"m": 0, "c": 0},
    {"m": 0, "c": 0},
    {"m": 0, "c": 0},
]

In [5]:
iterations = 10

In [6]:
N = 4
N

4

In [7]:
L = 0.05 # learning rate

In [8]:
def forward_node(node_id):
    params = node_params[node_id]
    x = node_data[node_id]["x"]
    return [params["m"] * i + params["c"] for i in x]

In [9]:
def loss_fn_node(node_id, y_pred):
    y = node_data[node_id]["y"]
    return (1 / N) * sum( (y[i] - y_pred[i]) ** 2 for i in range(N) )

In [10]:
def r2_score_node(node_id, y_pred):
    y = node_data[node_id]["y"]
    y_mean = node_y_mean[node_id]
    rss = sum( (y[i] - y_pred[i])**2 for i in range(N) )
    tss = sum( (y[i] - y_mean)**2 for i in range(N) )
    r2 = 1 - rss/tss 
    return r2

In [11]:
def gradient_m_node(node_id, y_pred):
    x = node_data[node_id]["x"]
    y = node_data[node_id]["y"]
    return (-2 / N) * sum( ( x[i] * (y[i] - y_pred[i]) ) for i in range(N) )

In [12]:
def gradient_c_node(node_id, y_pred):
    y = node_data[node_id]["y"]
    return (-2 / N) * sum( (y[i] - y_pred[i]) for i in range(N) )

In [13]:
for i in range(iterations):
    for node_id in range(len(node_data)):
        y_pred = forward_node(node_id)
        loss = loss_fn_node(node_id, y_pred)
        accuracy = r2_score_node(node_id, y_pred)
        print("Node:", node_id, "| Loss:", loss, "| Accuracy:", accuracy)
    
        # grad_m = gradient_m(y_pred)
        # grad_c = gradient_c(y_pred)
        # print("gradient of m:", grad_m, "| graident of c:", grad_c)
    
        # m = m - L * grad_m
        # c = c - L * grad_c
        # print("m:", m, "| c:", c)
        # print()

Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.5 | Accuracy: -169.0
Node: 2 | Loss: 132.5 | Accuracy: -529.0
Node: 0 | Loss: 2.5 | Accuracy: -9.0
Node: 1 | Loss: 42.